In [ ]:
import json
import csv
import re
import sys 
import requests
reload(sys)
sys.setdefaultencoding('UTF-8')
import string


#Definitions
commSite = 'http://www.dailymail.co.uk/reader-comments/p/asset/readcomments/'
start = '?max=100&offset='
end ='&order=desc&rcCache=shout'
headers = 'id', 'dateCreated', 'message', 'assetId', 'assetUrl','assetCommentCount', 'userAlias','userLocation','assetHeadline',   'userIdentifier', 'voteRating', 'voteCount', 'hasProfilePicture'
headersReq = {
    'Host': 'www.dailymail.co.uk',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
}


#Precompile RegExs
timeRegex = re.compile(r"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?Z")
pubRegex = re.compile(r"<meta property =\"article:published_time\" content=\"(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):.+?")


#UI
URL = raw_input('Enter Full Daily Mail Article URL: ')
article = re.search('\/article-(\d+)\/',URL).group(1)


#Find Publish date
req = requests.get(URL,headers=headersReq)
lines = req.text
match = re.search(pubRegex,lines)
if match:
    pubtime=match.groups()
pubTime = [match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)]
    

#Setup CSV
outfile  = open(article+'.csv', "wb")
writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
writer.writerow(headers+('wordCount','artCreatedDate','CommentPubAfter(Mins)','AverageWordCount'))


#First set of comments
reqURL = commSite + article
req = requests.get (reqURL, headers = headersReq)
html_source = req.text


numOfComments = re.findall ('{"total":(\d+)',html_source)
numOfParents = re.findall (',\"parentCommentsCount":(\d+),',html_source)
nums = int(numOfComments[0])
nump = int(numOfParents[0])


#Get rest of comments
messagesLen=[]
n=2
for i in range (0, nums,100): 
    comment = []
    req = requests.get (reqURL + start + str(i) + end, headers = headersReq)
    data = json.loads(req.text)
    for i in range(len(data['payload']['page'])):
        for header in headers:
            if header == 'dateCreated':
                match = re.search(timeRegex, (data['payload']['page'][i]['dateCreated']))
                if match:
                    (data['payload']['page'][i]['dateCreated']) = match.group(3)+'/'+match.group(2)+'/'+match.group(1)+' '+match.group(4)+':'+match.group(5)
            comment.append(data['payload']['page'][i].get(header,'N\\A'))
            if header == 'message':
                words = str(data['payload']['page'][i]['message'])
                words = words.translate(None, string.punctuation)
                words = re.split(r'[^0-9A-Za-z]+',words.strip())
                wordsCount=str(len(words)).split()
                messagesLen.append(len(words))
            delta = []
        delta = ['=' + '(' + 'B' + str(n) + '-' + 'O' + str(n) + ')' + '*1440'] #MS Excel format
        averageWordCount = ['=average($N$2' + ':' + 'N' + str(nump+1) + ')']
        comment=comment+wordsCount+pubTime+delta + averageWordCount
        writer.writerow(comment)
        comment = []
        n+=1
        
       
outfile.close()
      
req.connection.close()

print 'Done!'